In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'frn'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [2]:
#setting up path/ directory
#access specific file for specific participant
def load_epochs(pp_num, root_dir, erp_path):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'mra_p%03d_run0-epo.fif' % pp_num)

    epochs = mne.read_epochs(filename)
    #baseline for each trial
    if erp_path == 'frn':
        epochs = epochs.apply_baseline(baseline = (-1.5, -1.2))
    elif erp_path == 'lrp':
        epochs = epochs.apply_baseline(baseline = (-1.3, -1.0))
    elif erp_path == 'ern':
        epochs = epochs.apply_baseline(baseline = (-1.35, -1.05))

    return epochs, pp_directory

In [3]:
#get index of trials from behavioral data
#function will read in aligned data only
def epoch_index(pp):
    
    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\ALIGNED_learningcurve_degrees.csv')
    
    ppdata = list(df.iloc[:,pp+1])
    
    index = [i for i,v in enumerate(ppdata)]
    
    #epochs_out = epochs[index]
    numtrials = len(index)
    
    #return epochs_out, numtrials
    return index, numtrials

In [4]:
def error_index(task, errsize, pp):
    
    if task == 'rdmrot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rdmrot_ErrorSize_index.csv')
    elif task == 'rdmmir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rdmmir_ErrorSize_index.csv')
    elif task == 'rot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\rot_ErrorSize_index.csv')
    elif task == 'mir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\mir_ErrorSize_index.csv')
    
    ppdata = list(df.iloc[:,pp+1])

    index = [i for i,v in enumerate(ppdata) if errsize == v]

    numtrials = len(index)
    
    return index, numtrials

In [5]:
#separate function to get indexed epochs for random, since we have them before rotation or mirror
def trial_list_rdm(task, pp, index):
    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rdmrot':
            #get trial after first task (trial 48 to 95, 234 to 281)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
                    
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
    elif pp % 2 == 1:   
        if task == 'rdmrot':
            #get trial after first five tasks (trial 234 to 281)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

    
    return(triallist)
    

In [6]:
#get matching trials from eeg data, where we account for missing triggers
def epoch_trials(task, pp, epochs, index):

    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    if task == 'aln':
        #get trial numbers where tasktrig is 16149 and trigidx is 1
        tasksdf = df[0:48]
        subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16149]
        triallist = []
        for i in index:
            ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
            nint = int(ndat['trigidx'])
            if nint == 1:
                trial = int(ndat['ctrial'])
                triallist.append(trial)

        epochs = epochs[triallist]
    
    #for the other tasks, the order depends on their pp number
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rot':
            #rot trials are after first two tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
        elif task == 'mir':
            #get trial after first task (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
    elif pp % 2 == 1:
        if task == 'rot':
            #get trial after first five tasks (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
        elif task == 'mir':
            #get trial after first five tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
    
    return(epochs)

In [7]:
#save epochs
def output_condition_epochs(pp_num, data, ppdir, condname, err, task):
    # Save the cleaned data :)
    out_fname = os.path.join(ppdir, 'p%03d_SmallLarge_%s_%s_%s-epo.fif' % (pp_num, condname, err, task))
    data.save(out_fname, overwrite = True)

In [8]:
#save as an evoked object
def output_condition_evoked(pp_num, data, ppdir, condname, err, task):
    evoked = data.average()
    out_fname = os.path.join(ppdir, 'p%03d_SmallLarge_%s_%s_%s-ave.fif' % (pp_num, condname, err, task))
    evoked.save(out_fname, overwrite = True)

In [9]:
#save as evoked objects
#each perturbation condition can have separate loops
l_cutoff = 30

# ALIGNED
for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    idx, numtrials = epoch_index(pp)
    aldata = epoch_trials('aln', pp, data, idx)
    #print(pp, numtrials)
    output_condition_epochs(pp, aldata, ppdir, 'P3', 'SL', 'aligned')
    #filter at 30 Hz
    aldata = aldata.filter(l_freq= None, h_freq=l_cutoff)
    #then average across trials
    output_condition_evoked(pp, aldata, ppdir, 'P3', 'SL', 'aligned')

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\mra_p000_run0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\frn\mra_p001_run0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available


[Parallel(n_jobs=1)]: Done 3072 out of 3072 | elapsed:    1.0s finished


Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



...

[Parallel(n_jobs=1)]: Done 3072 out of 3072 | elapsed:    1.0s finished


Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.


[Parallel(n_jobs=1)]: Done 3072 out of 3072 | elapsed:    1.0s finished


In [10]:
#get grand averages - ALIGNED
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
#aligned

flist = []
evoked_list = []
for pp in participants:
    id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
    pp_directory = os.path.join(id_directory, erps)
    fname = os.path.join(pp_directory, 'p%03d_SmallLarge_%s_%s_%s-ave.fif' % (participants[pp], 'P3', 'SL', 'aligned'))
    evoked = mne.read_evokeds(fname)
    flist.append(evoked)
    evoked = flist[pp][0]
    evoked_list.append(evoked)
aln_grand = mne.grand_average(evoked_list)
aln_flist = evoked_list

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\p000_SmallLarge_P3_SL_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\frn\p001_SmallLarge_P3_SL_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\frn\p002_SmallLarge_P3_SL_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.

        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p020\frn\p020_SmallLarge_P3_SL_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\frn\p021_SmallLarge_P3_SL_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self

In [11]:
#save as evoked objects
errsizes = ['sml', 'lrg']
tasks = ['rot', 'mir']
l_cutoff = 30

# ROTATED AND MIRROR TRAINING

for task in range(0, len(tasks)):
    for pp in participants:
        data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
        for size in range(0, len(errsizes)):
            erridx, errtrials = error_index(tasks[task], errsizes[size], pp)
            if len(erridx) == 0:
                break
            #print(erridx)
            pdata = epoch_trials(tasks[task], pp, data, erridx)
            #print(pdata)
            output_condition_epochs(pp, pdata, ppdir, 'P3', errsizes[size], tasks[task])
            #filter at 30 Hz
            pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
            #then average across trials
            output_condition_evoked(pp, pdata, ppdir, 'P3', errsizes[size], tasks[task])

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\mra_p000_run0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 2304 out of 2304 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\frn\mra_p001_run0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 2304 out of 2304 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.


[Parallel(n_jobs=1)]: Done 2304 out of 2304 | elapsed:    0.7s finished


In [12]:
#get grand averages - ROTATED AND MIRROR
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
tasks = ['rot', 'mir']
errsizes = ['sml', 'lrg']
#rot and mir
for task in range(0, len(tasks)):
    for size in range(0, len(errsizes)):
        flist = []
        evoked_list = []
        for pp in participants:
            id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            pp_directory = os.path.join(id_directory, erps)
            fname = os.path.join(pp_directory, 'p%03d_SmallLarge_%s_%s_%s-ave.fif' % (participants[pp], 'P3', errsizes[size], tasks[task]))
            if os.path.exists(fname) == False:
                continue
            evoked = mne.read_evokeds(fname)
            flist.append(evoked)
            evoked = evoked[0]
            evoked_list.append(evoked)
        if task == 0 and size == 0:
            rot_sml_grand = mne.grand_average(evoked_list)
            rot_sml_flist = evoked_list
        elif task == 0 and size == 1:
            rot_lrg_grand = mne.grand_average(evoked_list)
            rot_lrg_flist = evoked_list
        elif task == 1 and size == 0:
            mir_sml_grand = mne.grand_average(evoked_list)
            mir_sml_flist = evoked_list
        elif task == 1 and size == 1:
            mir_lrg_grand = mne.grand_average(evoked_list)
            mir_lrg_flist = evoked_list

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\p000_SmallLarge_P3_sml_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\frn\p001_SmallLarge_P3_sml_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\frn\p002_SmallLarge_P3_sml_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...   

No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p020\frn\p020_SmallLarge_P3_sml_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\frn\p021_SmallLarge_P3_sml_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F

    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p008\frn\p008_SmallLarge_P3_lrg_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p009\frn\p009_SmallLarge_P3_lrg_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 1

Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p027\frn\p027_SmallLarge_P3_lrg_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p028\frn\p028_SmallLarge_P3_lrg_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p029\frn\p029_SmallLarge_P3_lrg_rot-ave

        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p015\frn\p015_SmallLarge_P3_sml_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p016\frn\p016_SmallLarge_P3_sml_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for th

Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\frn\p002_SmallLarge_P3_lrg_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p003\frn\p003_SmallLarge_P3_lrg_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p004\frn\p004_SmallLarge_P3_lrg_mir-ave

        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p022\frn\p022_SmallLarge_P3_lrg_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p023\frn\p023_SmallLarge_P3_lrg_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 36 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_p

In [13]:
#save as evoked objects
l_cutoff = 30

#RANDOM

errsizes = ['sml', 'lrg']
            
# RANDOM
for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    for size in range(0, len(errsizes)):
        idx, numtrials = error_index('rdmrot', errsizes[size], pp)
        rdmrot_trials = trial_list_rdm('rdmrot', pp, idx)
        print(rdmrot_trials)
        idx, numtrials = error_index('rdmmir', errsizes[size], pp)
        rdmmir_trials = trial_list_rdm('rdmmir', pp, idx)
        #print(rdmmir_trials)
        if size == 0:
            rdm_sml_trials = rdmrot_trials + rdmmir_trials
            rdm_sml_trials.sort()
            pdata = data[rdm_sml_trials]
        elif size == 1:
            rdm_lrg_trials = rdmrot_trials + rdmmir_trials
            rdm_lrg_trials.sort()
            pdata = data[rdm_lrg_trials]
        #print(pdata)
        output_condition_epochs(pp, pdata, ppdir, 'P3', errsizes[size], 'rdm')
        #filter at 30 Hz
        pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
        #then average across trials
        output_condition_evoked(pp, pdata, ppdir, 'P3', errsizes[size], 'rdm')

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\mra_p000_run0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
[52, 57, 59, 62, 63, 64, 66, 68, 69, 70, 71, 74, 75, 76, 80, 81, 82, 84, 89, 94, 95]
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


[48, 49, 51, 54, 55, 56, 58, 60, 61, 67, 73, 78, 79, 83, 85, 86, 87, 90, 91, 92, 93]
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 2688 out of 2688 | elapsed:    0.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\frn\mra_p001_run0-epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available


...

[Parallel(n_jobs=1)]: Done 2688 out of 2688 | elapsed:    0.8s finished


    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
[235, 241, 242, 243, 244, 246, 247, 248, 250, 254, 260, 261, 264, 268, 270, 272, 274, 276, 277, 279, 281]
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


[234, 237, 238, 239, 240, 245, 252, 253, 255, 256, 257, 259, 262, 263, 265, 266, 267, 271, 273, 275, 278]
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 89 samples (0.445 sec)



[Parallel(n_jobs=1)]: Done 2688 out of 2688 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 2688 out of 2688 | elapsed:    0.8s finished


In [14]:
#get grand averages - RANDOM
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
tasks = ['rdm']
errsizes = ['sml', 'lrg']

#rdm
for task in range(0, len(tasks)):
    for size in range(0, len(errsizes)):
        flist = []
        evoked_list = []
        for pp in participants:
            id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            pp_directory = os.path.join(id_directory, erps)
            fname = os.path.join(pp_directory, 'p%03d_SmallLarge_%s_%s_%s-ave.fif' % (participants[pp], 'P3', errsizes[size], tasks[task]))
            if os.path.exists(fname) == False:
                continue
            evoked = mne.read_evokeds(fname)
            flist.append(evoked)
            evoked = evoked[0]
            evoked_list.append(evoked)
        if task == 0 and size == 0:
            rdm_sml_grand = mne.grand_average(evoked_list)
            rdm_sml_flist = evoked_list
        elif task == 0 and size == 1:
            rdm_lrg_grand = mne.grand_average(evoked_list)
            rdm_lrg_flist = evoked_list

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\p000_SmallLarge_P3_sml_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 42 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\frn\p001_SmallLarge_P3_sml_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 42 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\frn\p002_SmallLarge_P3_sml_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...   

No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p020\frn\p020_SmallLarge_P3_sml_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 42 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\frn\p021_SmallLarge_P3_sml_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 42 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F

    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 42 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p008\frn\p008_SmallLarge_P3_lrg_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 42 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p009\frn\p009_SmallLarge_P3_lrg_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 42 - aspect type = 1

Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p027\frn\p027_SmallLarge_P3_lrg_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 42 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p028\frn\p028_SmallLarge_P3_lrg_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 42 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p029\frn\p029_SmallLarge_P3_lrg_rdm-ave

In [15]:
#convert evokeds to data frame, so we can plot the ERPs better

flists = ['aln', 
          'rot_sml', 'rot_lrg',
          'mir_sml', 'mir_lrg',
          'rdm_sml', 'rdm_lrg']

for f in flists:
    newdf = pd.DataFrame()
    for pp in participants:
        if f == 'aln':
            df = aln_flist[pp].to_data_frame()
        elif f == 'rot_sml':
            df = rot_sml_flist[pp].to_data_frame()
        elif f == 'rot_lrg':
            df = rot_lrg_flist[pp].to_data_frame()
        elif f == 'mir_sml':
            df = mir_sml_flist[pp].to_data_frame()
        elif f == 'mir_lrg':
            df = mir_lrg_flist[pp].to_data_frame()
        elif f == 'rdm_sml':
            df = rdm_sml_flist[pp].to_data_frame()
        elif f == 'rdm_lrg':
            df = rdm_lrg_flist[pp].to_data_frame()
             
        time = df['time'].tolist()
        df = df[['FCz', 'Fz', 'Cz', 'Pz']]
        #df = df[['C3', 'C4']]
        ppdat = []
        for timept in range(0,800):
            subdf = df.loc[timept,:].mean()
            ppdat.append(subdf)
    
        newdf['pp%03d'% pp] = ppdat
    
    newdf['time'] = time
    newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_SmallLarge_%s_P3.csv' % (f))
    newdf.to_csv(newdf_filename)

In [16]:
#plot FRN/ERN here using MNE functions
#channels = ['FCz', 'Fz']
channels = ['FCz', 'Fz', 'Cz', 'Pz']
#channels = ['C3']
#channels = ['C4']
# channels = ['Oz','O1','O2']
evokeds = dict(aln = aln_flist, 
               rot_sml = rot_sml_flist,
               rot_lrg = rot_lrg_flist, 
               mir_sml = mir_sml_flist, 
               mir_lrg = mir_lrg_flist,
               rdm_sml = rdm_sml_flist,
               rdm_lrg = rdm_lrg_flist)
#plot
mne.viz.plot_compare_evokeds(evokeds, picks=channels, combine = 'mean', legend = 'lower right', ylim=dict(eeg=[-15, 5]), ci=True)

combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"


[<Figure size 800x600 with 1 Axes>]